<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%9E%AD%ED%82%B9_%EB%89%B4%EC%8A%A4_%EC%8A%A4%ED%81%AC%EB%9E%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 파이썬 웹 요청 라이브러리
import requests

In [3]:
## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 네이버 랭킹 뉴스 웹페이지 요청. 성공하면 응답이옴(r)
r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)

In [31]:
## 응답 정보에는 많은 게 있지만 우린 text가져가면 된다. text는 단순 문자열이다.
ranking_news_html = r.text


In [ ]:
## html 문서 -> 문자열(태그 접근 X)

html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

html_doc

In [ ]:
## 단순 문자열을 태그를 기준으로 트리 형식으로 저장해주는 라이브러리
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')


# select : 원하는 태그를 다수 선택
# select_one : 원하는 태그를 한개만 선택
soup.select_one('p.title')


# 태그.text : 태그로 감싸여진 문자 텍스트값 가져오기
# 태그[속성명] : 속성명에 해당하는 값을 가져오기
soup.select_one('title').text
soup.select_one('a#link2').text

atag = soup.select_one('a#link1')

print(atag.text, atag['href'])

# 주요 선택자
## 태그이름 (태그명)
## 클래스이름 (.클래스명)
## 아이디이름 (#아이디명)



In [ ]:
## 네이버 랭킹 뉴스 페이지 태그 문서 트리 구조로 파싱
soup = BeautifulSoup(ranking_news_html, 'html.parser')
soup

In [36]:
# 원하는 정보가 있는 태그가 페이지 전체에 걸쳐 방대하게 펼쳐저 있기 때문에
# 탐색 범위를 좁히기 위해 적절한 부분으로 자른다.
news_list = soup.select("div.rankingnews_box")

In [39]:
box = news_list[0] # 페이지에 존재하는 여러 div.rankingnews_box 중 가장 첫번째 결과
# 여기서 div.rankingnews_box는 각 언론사를 의미한다.

In [42]:
box.select_one('strong.rankingnews_name').text # 언론사의 이름을 찾아서 텍스트를 꺼내온다.

'서울경제'

In [43]:
box2 = news_list[1] # 두번째 언론사 태그
box2.select_one('strong.rankingnews_name').text # 두번째 언론사의 언론사 이름

'헤럴드경제'

In [ ]:
# news_list 는 82개의 언론사 태그가 담겨져 있는 리스트
for box in news_list: # 각 언론사 태그를 순차적으로 꺼낸다

  journal_name = box.select_one('strong.rankingnews_name').text # 언론사의 이름 꺼낸다

  if journal_name == 'SBS' : # 언론사 이름이 SBS 라면

    news_li = box.select("li") # 해당 언론사 태그 내에서 li 태그들을 찾는다. 5개의 li태그를 찾을 수 있다.
    target = news_li[0] # li 태그 중 첫번째 태그를 꺼내서 target변수에 담는다. 여기서 li는 해당 언론사의 상위 5개 랭킹 뉴스 태그이다.
    news_a = target.select_one('div.list_content a') # 첫번째 랭킹뉴스 태그를 꺼내서 news_a 변수에 담는다.

    dic = {
        'title' : news_a.text, # 첫번째 랭킹 뉴스의 제목을 꺼낸다.
        'link' : news_a['href'] # 첫번째 랭킹 뉴스의 본문 링크를 꺼낸다.
    }

# 최종 파이썬 딕셔너리로 만든다. 이후 json을 이용해 파일 저장하면 된다.
print(dic)



In [56]:
# 네이버 TV TOP 100 정보 가져오기

## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 네이버 TV TOP 100 웹페이지 요청. 성공하면 응답이옴(r)
r = requests.get('https://tv.naver.com/r', headers=headers)


In [ ]:
mon_webtoon_html = r.text
## 네이버 TV TOP 100 페이지 태그 문서 트리 구조로 파싱
soup = BeautifulSoup(mon_webtoon_html, 'html.parser')
soup

In [ ]:
# soup.select_one('div.cds_area')
cds_list = soup.select('div.cds') # 태그명이 div이고 class명이 cds인 태그를 여러개 선택



In [72]:
cds_list[0].select('dd.chn') # 가져온 태그들 중 가장 첫번째 태그 선택




[<dd class="chn">
 <a href="https://tv.naver.com/npop" onclick="clickcr(this, 'top.chtitle', '', '4', event)">NPOP (엔팝)</a>
 </dd>]

In [77]:
atag = cds_list[0].select_one('dt.title a') # 해당 태그에서 태그 이름이 dt 이고 class이름이 title인 태그 아래에 있는 a 태그 선택
atag['title'] # a 태그의 속성 중 title 속성의 값을 꺼낸다.

'[MAIN TRAILER] NPOP, WANNA JOIN? Ι 9/6(수) 8PM NPOP EP.01'